In [1]:
import os
import matplotlib.pyplot as plt 
%matplotlib inline
from keras.callbacks import ModelCheckpoint

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, Flatten, MaxPooling2D, Reshape
from keras import optimizers

LEARNING_RATE = 1e-4

model = Sequential()
model.add(Convolution2D(32, 8, 8, subsample=(4, 4), border_mode='same',input_shape=(80,80,1)))  #80*80*4
model.add(Activation('relu'))
model.add(Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(3))

adam = optimizers.Adam(lr=LEARNING_RATE)
model.compile(loss='mse',optimizer=adam)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 20, 20, 32)        2080      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        32832     
_________________________________________________________________
activation_2 (Activation)    (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 64)        36928     
_________________________________________________________________
activation_3 (Activation)    (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
__________

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (8, 8), input_shape=(80, 80, 1..., strides=(4, 4), padding="same")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 4), strides=(2, 2), padding="same")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), strides=(1, 1), padding="same")`
  del sys.path[0]


In [3]:
def load_model():
  model.load_weights("model.h5")
  adam = optimizers.Adam(lr=LEARNING_RATE)
  model.compile(loss='mse',optimizer=adam)

load_model()

### Con entrenamiento dentro de deque()

In [4]:
mode = 'Run'
#mode = 'observe'

import random 
import gym 
from collections import deque
import numpy as np
import json 
import random
from skimage import color
from skimage import transform 
from skimage import exposure
import time 

def preProcessImage(img):
    Img = color.rgb2gray(img)
    Img = transform.resize(Img,(80,80))
    return Img 
  

# Pong es el juego 
env = gym.make('Pong-v0')
ob = env.reset()  

# accion inicial random  
#a_t = random.choice([0,2,3])

# accion inicial en Pong 

ACTIONS = 3 # number of valid actions

P_ACTIONS = [0,2,3]

GAMMA = 0.95 # decay rate of past observations
OBSERVATION = 300 # timesteps to observe before training
EXPLORE = 100000 # frames over which to anneal epsilon

FINAL_EPSILON = 0.1 # final value of epsilon
INITIAL_EPSILON = 0.5 # starting value of epsilon

REPLAY_MEMORY = 300 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAME_PER_ACTION = 1

img_rows , img_cols = 80, 80
#Convert image into Black and white
img_channels = 4 #We stack 4 frames



def trainNetwork(model,mode):
    # open up a game state to communicate with emulator
    env = gym.make('Pong-v0')
    
    ob = env.reset() 

    # store the previous observations in replay memory
    D = deque()
    
    env.render()
    
    # get the first state by doing nothing and preprocess the image to 80x80x4
    #do_nothing = np.zeros(ACTIONS)
    do_nothing = 0
    #x_t, r_0, terminal = game_state.frame_step(do_nothing)
    x_t, r_0, terminal, _ = env.step(do_nothing) # porque la accion 0 es 0 y es hacer nada
    x_t = preProcessImage(x_t) # modifica tamaño y grey de la imagen
    
    # normaliza imagen 
    x_t = exposure.rescale_intensity(x_t, out_range=(0, 255))

    x_t = x_t / 255.0

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
    #print (s_t.shape)
    
    #In Keras, need to reshape
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*80*80*4

    if mode == 'Run':
        OBSERVE = 999999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weights")
        #model.load_weights("model.h5")
        #adam = optimizers.Adam(lr=LEARNING_RATE)
        #model.compile(loss='mse',optimizer=adam)
        print ("Weights loaded successfully. En realidad no lo cargamos  y usamos el mismo") 
        
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = INITIAL_EPSILON
    
    reward_acumulado = 0 
    t = 0
    
    opcion = ''
    
    while (t <= OBSERVATION + EXPLORE):
        loss = 0
        Q_sa = 0
        r_t = 0
        #a_t = np.zeros([ACTIONS])
        a_t = 0 
        
        
        #choose an action epsilon greedy
        if t % FRAME_PER_ACTION == 0:
          if random.random() <= epsilon:
            opcion = 'random'
            #print("----------Random Action----------")
            a_t = random.choice([0,1,2])
            
          else:
            #print("++++++++++Predicted Action----------")
            q = model.predict(s_t)     #input a stack of 4 images, get the prediction
            max_Q = np.argmax(q)
            opcion = 'prediccion'
            a_t = max_Q
            

        #We reduced the epsilon gradually
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        #run the selected action and observed next state and reward
        x_t1_colored, r_t, terminal,_ = env.step(P_ACTIONS[a_t]) # decia env.frame
        
        # escala de grises y resize 
        x_t1 = preProcessImage(x_t1_colored)
        
        # normaliza imagen 
        x_t1 = exposure.rescale_intensity(x_t1, out_range=(0, 255))
        x_t1 = x_t1 / 255.0
        
                      
        # reshape keras 
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x80x80x1
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3)

        # store the transition in D
        D.append((s_t, a_t, r_t, s_t1, terminal))
        
        #revisar esto dl reward
        reward_acumulado += r_t 
                                            
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE:
            
            minibatch = random.sample(D,BATCH)
            inputs = np.zeros((BATCH,s_t.shape[1],s_t.shape[2],s_t.shape[3]))
            targets = np.zeros((inputs.shape[0],3))
       
            state_t, action_t, reward_t, state_t1, terminal_t1 = zip(*minibatch)
            
            #print('TIME STEP: ', t, 'largo de D: ', len(D))

            state_t = np.concatenate(state_t)
            state_t1 = np.concatenate(state_t1)
                      
            targets = model.predict(state_t)
            
            print("--------- Targets step: ",t)
            print(targets)
            print("-------------------------")
              
            Q_sa = model.predict(state_t1)
            
            if terminal_t1:
              targets[range(BATCH), action_t] = reward_t
              print('suma reward', reward_t)
              
            else:
              targets[range(BATCH), action_t] = reward_t + GAMMA*np.max(Q_sa)
              
            
            #targets = [P_ACTIONS[np.argmax(x)] for x in targets]
              
                        
            loss += model.train_on_batch(state_t, targets)

            #print('entrenando*****, loss: ', loss, '************************')

        s_t = s_t1
        t = t + 1

        # save progress every 10000 iterations
        if t % 50000 == 0:
            print("Now we save model")
            model.save("model.h5", overwrite=True)
                
            # guardamos modelo en el drive 
            save_to_drive('model.h5')

        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        #print("TIMESTEP", t, "/ STATE", state, \
         #   "/ EPSILON", epsilon, "/ ACTION", a_t, "/ REWARD", r_t, "/ R_ACUM", reward_acumulado, \
          #  "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)
        
        if terminal==True :
          print(t,' - reward acumulado', reward_acumulado, 'STATE: ', state, 'EPSILON', epsilon, 'LAST ACTION', a_t, 'OPCION', opcion)
          reward_acumulado = 0
          terminal = False
          ob = env.reset()
          
    print("Episode finished!")
    print("************************")

## NUEVO .ENFOQUE

In [5]:
import random 
import gym 
from collections import deque
import numpy as np
import json 
import random
from skimage import color
from skimage import transform 
from skimage import exposure
import time 
from matplotlib import pyplot as plt

def preProcessImage(img):
    Img = color.rgb2gray(img)
    Img = transform.resize(Img,(80,80))
    return Img 

def aplicar_maxpool(matrix):
  lala = matrix.reshape(1,80,80,4) #le hace reshape al vector concadenado
  lala = np.amax(lala, axis=3) # calcula el maximo
  lala = lala.reshape(1,80,80,1) #le hace reshape al vector concadenado
  #lala = lala.reshape(80,80) #lo deja como imagen de 80x80 (no se usa porque keras recibe 1x80x80x1)
  #plt.imshow(lala, interpolation='nearest')
  #plt.show()
  return lala

def mostrarImagen(imagen) : #la entrada debe tener un shape de 1x80x80x1
  lala = imagen.reshape(80,80) #lo deja como imagen de 80x80 (no se usa porque keras recibe 1x80x80x1)
  plt.imshow(lala, interpolation='nearest')
  plt.show()

def normalizar(numero):
  result = 0 
  if numero < 0:
    result = -0.8 
  else:
    result = 1 
  
  return result 
    
  

# Pong es el juego 
env = gym.make('Pong-v0')
ob = env.reset()  

ACTIONS = 3 # numero de acciones validas

P_ACTIONS = [0,2,3]

GAMMA = 0.5 # decay rate of past observations
OBSERVATION = 300 # timesteps to observe before training
EXPLORE = 250000 # frames over which to anneal epsilon

FINAL_EPSILON = 0.001 # final value of epsilon
INITIAL_EPSILON = .1 # starting value of epsilon

REPLAY_MEMORY = 300 # number of previous transitions to remember
BATCH = 10 # size of minibatch
FRAME_PER_ACTION = 4

img_rows , img_cols = 80, 80
#Convert image into Black and white
img_channels = 4 #We stack 4 frames


D = deque()


def trainNetwork2(model,epsilon,mode):
    reward_acum = 0
    loss = 0
    entrenando = ''
    env = gym.make('Pong-v0')
    ob = env.reset() 
  
    # store the previous observations in replay memory
    D.clear()
    D_por_episodio = list()
        
    last_4_frames = []

    for i in range(4):
      x_t, r_0, terminal, _ = env.step(0) # porque la accion 0 es 0 y es hacer nada
      x_t = preProcessImage(x_t) # modifica tamaño y grey de la imagen
      last_4_frames.append(x_t)
   
    s_t = np.stack((last_4_frames[0],last_4_frames[1], last_4_frames[2], last_4_frames[3]), axis=2)
    s_t = aplicar_maxpool(s_t) # S_T0 CONTIENE EL ESTADO INICIAL SUB 0 CON SHAPE 1x80x80x1
    
    lista_estados = []
    r_t1 = r_0
    for t in range (EXPLORE) :
        
        env.render()
        
        time.sleep(0.1)
        
        #SELECCIONAR UNA ACCION ALEATORIA CON PROBABILIDAD EPSILON
        if random.random() <= epsilon:
          opcion = 'random'
          a_t = random.choice([0,1,2])
            
        else:
          opcion = 'prediccion'
          q = model.predict(s_t)    
          max_Q = np.argmax(q)
          
          a_t = max_Q
        #JUEGA 4 VECES CON LA MISMA ACCION
        r_t1 = 0
        r_instanstanteo_4 = 0
        for it in range(4):
          x_t1, r_t1, terminal1, _ = env.step(P_ACTIONS[a_t])
          x_t1 = preProcessImage(x_t1)
          
          lista_estados.append(x_t1)
          
          if r_t1 != 0:
            r_instanstanteo_4 = r_t1
          #r_t1 = max(0,r_t1)
          
          if r_t1 >= 0:
              reward_acum += r_t1
        
        r_t1 = r_instanstanteo_4   
          # si entre los 4 juegos uno de ellos es terminal, se sale del juego y se queda con lo que encontro hasta ahi
        
                    
        s_t1 = np.stack(lista_estados, axis=2)
        s_t1 = aplicar_maxpool(s_t1)
        lista_estados=[]
        
        #mostrarImagen(s_t1)
        
        D_por_episodio.append((s_t, a_t, r_t1, s_t1, terminal1))
        s_t = s_t1
        r_t = r_t1
        
        # REVISAR ESTO PORQUE EL EPSILON SE DISMINUYE 1 VEZ CADA t, PERO LA IMAGEN LA AGREGA CADA 4 t
        if epsilon > FINAL_EPSILON and t > OBSERVATION:
          epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / 1000000
        
        # si es terminal, los agrega a la deque desde la lista
        # -------------------------------------------------------
        if r_t != 0:
            
            if r_t > 0:
            
                for D_list in D_por_episodio:
                    DDs_t, DDa_t, DDr_t, DDs_t1, DDterminal = D_list
                    D.append((DDs_t, DDa_t, 4*r_t, DDs_t1, DDterminal)) # lo agrega a la memory replay con reward R_T (reward final)
            
            else:
                
                for D_list in D_por_episodio:
                    DDs_t, DDa_t, DDr_t, DDs_t1, DDterminal = D_list
                    D.append((DDs_t, DDa_t, 4*r_t, DDs_t1, DDterminal)) # lo agrega a la memory replay con reward R_T (reward final)
            
            
            if len(D) > REPLAY_MEMORY:
              D.popleft()
              
            D_por_episodio.clear()
        
        #NUESTRO TIEMPO DE OBSERVACION ES CUANDO SE LLENA LA REPLAY MEMORY, UNA VEZ LLENA COMIENZA A ENTRENAR
        if len(D) >= REPLAY_MEMORY :
            minibatch = random.sample(D,BATCH)
            inputs = np.zeros((BATCH,s_t.shape[1],s_t.shape[2],s_t.shape[3]))
            targets = np.zeros((inputs.shape[0],3))
            state_t, action_t, reward_t, state_t1, terminal_t1 = zip(*minibatch)
            state_t = np.concatenate(state_t)
            state_t1 = np.concatenate(state_t1)
                      
            targets = model.predict(state_t)
            
            Q_sa = model.predict(state_t1)
            
            
            if terminal_t1 == True: #terminal_t1 es el terminal del minibatch
              targets[range(BATCH), action_t] = reward_t
              print('suma reward', reward_t)
              
            else:
              targets[range(BATCH), action_t] = [x + normalizar(GAMMA*np.max(Q_sa)) for x in reward_t]
              
              #print([x + normalizar(GAMMA*np.max(Q_sa)) for x in reward_t])
              
              
            loss += model.train_on_batch(state_t, targets)
            entrenando = 'entrenando'
        
        # terminal1 es el terminal del juego (lo que acaba de jugar)
        if terminal1==True :
          print("ACABO LA PARTIDA CON REWARD ", reward_acum)
          print(t,'Size DEQUE:', len(D),' - Action:', a_t,'- Reward_inst:',r_t1, ' - Reward acum:', reward_acum, ' - Rand_or_pred:',opcion,entrenando, 'EPSILON', epsilon)
          terminal = False
          reward_acum = 0
          env.reset()
          #time.sleep(10)
        t += 1
        
        # guardamos el modelo cada 50000 iteraciones 
        if t % 50000 == 0:
            print("Now we save model")
            model.save("model.h5", overwrite=True)
                
            # guardamos modelo en el drive 
            #save_to_drive('model.h5')
            
        print(t,'Size DEQUE:', len(D),' - Action:', a_t,'- Reward_inst:',r_t1, ' - Reward acum:', reward_acum, ' - Rand_or_pred:',opcion,entrenando)
        #mostrarImagen(s_t1)
        
    return epsilon
